In [1]:
# Translated to .py by Marti Montesinos
# Modified by Peiyu Jing

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_TIME	 = Beta('B_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)

# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
            + B_GA * GA
SM_SP = ASC_SM * one + B_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
            + B_SENIOR * SENIOR
V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)

# Estimation of the model
logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "MNL_SM_socioec"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-0.608
ASC_SM=	-0.135
B_CAR_COST=	-0.00936
B_GA=	0.557
B_HE=	-0.00586
B_SENIOR=	-1.88
B_SM_COST=	-0.0104
B_TIME=	-0.0111
B_TRAIN_COST=	-0.0268
                 Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR      -0.608440  0.121568  -5.004917  5.588601e-07      0.143478   
ASC_SM       -0.134683  0.084410  -1.595589  1.105806e-01      0.106487   
B_CAR_COST   -0.009360  0.000906 -10.325863  0.000000e+00      0.001167   
B_GA          0.556606  0.187657   2.966086  3.016162e-03      0.191098   
B_HE         -0.005863  0.001048  -5.595495  2.199930e-08      0.001056   
B_SENIOR     -1.878375  0.115885 -16.209010  0.000000e+00      0.108531   
B_SM_COST    -0.010432  0.000550 -18.951004  0.000000e+00      0.000744   
B_TIME       -0.011125  0.000613 -18.152705  0.000000e+00      0.001210   
B_TRAIN_COST -0.026813  0.001165 -23.018323  0.000000e+00      0.001760   

              Rob. t-test  Rob. p-value  
ASC_CAR         -4.240642  2.228812e-05  
ASC_SM       